In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
names = open('names.txt','r').read().splitlines()

In [3]:
chars1 = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars1)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
block_size = 3
X,Y = [],[]
for name in names :
    context = [0] * block_size
    name = name + '.'
    for ch in name :
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context),'--->',itos[ix])
        context = context[1:] + [ix];
X = torch.tensor(X)
Y = torch.tensor(Y)

In [5]:
def dataset(names) :
    block_size = 3
    X,Y = [],[]
    for name in names :
        context = [0] * block_size
        name = name + '.'
        for ch in name :
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context),'--->',itos[ix])
            context = context[1:] + [ix];
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

In [6]:
import random
random.seed(42)
random.shuffle(names)
n1 = int(0.8 * len(names))
n2 = int(0.9 * len(names))

Xtr,Ytr = dataset(names[:n1])
Xdev,Ydev = dataset(names[n1:n2])
Xtest,Ytest = dataset(names[n2:])

In [7]:
gen = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,10),generator = gen)

In [8]:
W1 = torch.randn((30,200),generator = gen) * 0.2
b1 = torch.randn(200,generator = gen) * 0.01
W2 = torch.randn((200,27),generator = gen)*0.01
b2 = torch.randn(27,generator = gen)*0

bngain = torch.ones((1,200)) 
bnbias = torch.zeros((1,200))

bmean = torch.zeros((1,200))
bstd = torch.ones((1,200)) 

parameters = [C,W1,W2,b1,b2,bngain,bnbias]

In [9]:
for p in parameters :
    p.requires_grad = True

In [10]:
for i in range(100000) :
    ix = torch.randint(0,Xtr.shape[0],(32,))
    hpreact = C[Xtr[ix]].view((-1,30)) @ W1 + b1
    bnmeani = hpreact.mean(0,keepdim = True)
    bnstdi = hpreact.std(0,keepdim = True)
    hpreact = bngain*(hpreact - bnmeani)/bnstdi + bnbias
    with torch.no_grad():
        bmean = 0.999*bmean + 0.1*bnmeani
        bstd = 0.999*bstd + 0.1*bnstdi
    h = torch.tanh(hpreact);
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Ytr[ix])
    for p in parameters :
        p.grad = None
    loss.backward()
    lr = 0.1 if i <= 80000 else 0.01
    for p in parameters :
        p.data -= lr*p.grad
print(loss.item())

1.999336838722229


In [11]:
# with torch.no_grad():
#     emb = C[Xtr]
#     embcat = emb.view(emb.shape[0],-1)
#     hpreact = embcat @ W1 + b1

#     bmean = hpreact.mean(0,keepdim = True)
#     bstd = hpreact.std(0,keepdim = True)

In [12]:
with torch.no_grad():
    hpreact = C[Xdev].view((-1,30)) @ W1 + b1
    hpreact = bngain*(hpreact - bmean)/bstd + bnbias
    h = torch.tanh(hpreact);
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Ydev)
loss

tensor(4.1251)

In [13]:
with torch.no_grad():
    hpreact = C[Xtest].view((-1,30)) @ W1 + b1
    hpreact = bngain*(hpreact - bmean)/bstd + bnbias
    h = torch.tanh(hpreact);
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Ytest)
loss

tensor(4.1352)

In [14]:
block_size = 3
context = [0] * block_size
result = []
while True:
    hpreact = C[torch.tensor([context])].view((1,-1))@ W1 + b1
    hpreact = bngain*(hpreact - bmean)/bstd + bnbias
    h = torch.tanh(hpreact);
    layer2 = h @ W2 + b2
    probs = F.softmax(layer2,dim = 1)
    ix = torch.multinomial(probs,1,replacement = True,generator = gen).item()
    context = context[1:] + [ix]
    result.append(ix)
    if ix == 0 :
        break
print(''.join(itos[i] for i in result))

oyeiaieiyoauce.
